<a href="https://colab.research.google.com/github/Patelhlt/My-daily-work/blob/main/p1modeltraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Part 1: Setup and Installation**

Install the necessary libraries using pip:

In [ ]:
pip install nltk numpy tensorflow keras flask


In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
   

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('daily_dialog')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for daily_dialog contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/daily_dialog.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/11118 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# View the dataset structure
print(dataset)

# Access a specific split, e.g., 'train'
train_dataset = dataset['train']

# Display the first few examples
print(train_dataset[:5])


DatasetDict({
    train: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 11118
    })
    validation: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 1000
    })
})
{'dialog': [['Say , Jim , how about going for a few beers after dinner ? ', ' You know that is tempting but is really not good for our fitness . ', ' What do you mean ? It will help us to relax . ', " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ", " I guess you are right.But what shall we do ? I don't feel like sitting at home . ", ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ', " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ", ' Sounds great to me ! If they are willing , we could ask them to go dancin

In [ ]:
# Example preprocessing for a conversational dataset
def preprocess_examples(examples):
    # Assuming 'dialog' is the field containing conversations
    questions = []
    answers = []
    for dialog in examples['dialog']:
        for i in range(len(dialog) - 1):
            questions.append(dialog[i])
            answers.append(dialog[i + 1])
    return questions, answers

questions, answers = preprocess_examples(train_dataset)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

# Tokenize and pad sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + answers)

vocab_size = len(tokenizer.word_index) + 1
X = tokenizer.texts_to_sequences(questions)
y = tokenizer.texts_to_sequences(answers)

max_length = max(len(seq) for seq in X)
X = pad_sequences(X, maxlen=max_length, padding='post')
y = pad_sequences(y, maxlen=max_length, padding='post')

# Use a smaller subset of data (for demonstration purposes)
X_train = X[:1000]  # Use the first 1000 samples
y_train = y[:1000]

# Model training setup
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=64, input_length=max_length))  # Reduced output_dim
model.add(LSTM(64, return_sequences=True))  # Reduced LSTM units
model.add(Dense(vocab_size, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reduce the number of epochs and increase the batch size
model.fit(X_train, y_train, epochs=3, batch_size=100)  # Fewer epochs and larger batch size


Epoch 1/3
10/10 [==============================] - 6s 264ms/step - loss: 9.7798 - accuracy: 0.7543
Epoch 2/3
10/10 [==============================] - 3s 301ms/step - loss: 9.3456 - accuracy: 0.9468
Epoch 3/3
10/10 [==============================] - 3s 266ms/step - loss: 8.0193 - accuracy: 0.9468


In [ ]:
# Save the trained model
model.save('chatbot_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
